# SentimentalAgent / DebateAgent 스모크 테스트 노트북
이 노트북은 `SentimentalAgent` 단독 테스트와 `DebateAgent` 오케스트레이션 흐름( get_opinion → get_ensemble → run )을 빠르게 확인합니다.

## 사용법
1. 아래 **환경 설정** 셀에서 프로젝트 루트 경로가 잘 잡히는지 확인
2. **파라미터** 셀에서 티커/라운드/리빌드/프리트레인 설정
3. 순서대로 실행

> 주의: 이 노트북은 *외부 인터넷* 접속을 하지 않습니다. (yfinance는 로컬 환경에서 동작 가정)


In [ ]:

# === 환경 설정 ===
import os, sys, pathlib

# 현재 노트북 위치 기준으로 capstone_project 루트 추정
here = pathlib.Path.cwd()
candidates = [here, *here.parents]
root = None
for p in candidates:
    if (p / "agents").exists() and (p / "core").exists():
        root = p
        break

if root is None:
    # 수동 지정이 필요하면 아래 경로로 수정
    root = pathlib.Path.cwd()  # TODO: 필요시 변경
    print("[warn] 프로젝트 루트를 자동으로 찾지 못했습니다. 현재 경로를 루트로 사용합니다:", root)
else:
    print("[info] 프로젝트 루트:", root)

if str(root) not in sys.path:
    sys.path.insert(0, str(root))


In [ ]:

# === 파라미터 ===
TICKER = "NVDA"        # 테스트할 티커
ROUNDS = 1             # Debate 라운드 수
REBUILD = False        # 데이터셋 강제 재생성 여부
FORCE_PRETRAIN = False # 모델 사전학습 강제 여부

print("Params:", TICKER, ROUNDS, REBUILD, FORCE_PRETRAIN)


In [ ]:

# === 공용 유틸 ===
from pprint import pformat
from typing import Any, Dict

def safe_get(obj, name, default=None):
    return getattr(obj, name, default)

def print_kv(title: str, d: Dict[str, Any]):
    print(f"\n=== {title} ===")
    print(pformat(d, width=100))

def extract_opinion_dict(opinion_obj) -> Dict[str, Any]:
    # Opinion dataclass 형태 안전 추출
    if opinion_obj is None:
        return {}
    target = safe_get(opinion_obj, "target", None)
    d = {
        "agent_id": safe_get(opinion_obj, "agent_id", None),
        "reason": safe_get(opinion_obj, "reason", None),
        "target": {
            "next_close": safe_get(target, "next_close", None) if target else None,
            "uncertainty": safe_get(target, "uncertainty", None) if target else None,
            "confidence": safe_get(target, "confidence", None) if target else None,
            "feature_cols": safe_get(target, "feature_cols", None) if target else None,
        },
    }
    return d

def extract_target_dict(target_obj) -> Dict[str, Any]:
    return {
        "next_close": safe_get(target_obj, "next_close", None),
        "uncertainty": safe_get(target_obj, "uncertainty", None),
        "confidence": safe_get(target_obj, "confidence", None),
    }


In [ ]:

# === SentimentalAgent 단독 테스트 ===
from agents.sentimental_agent import SentimentalAgent

ag = SentimentalAgent(ticker=TICKER)
X = ag.searcher(ticker=TICKER)   # 데이터 준비 (없으면 자동 생성)
_ = ag.load_model()              # 모델 로드 (없으면 lazy 모델 유지)

# 예측
target = ag.predict(X)           # BaseAgent.predict -> Target(next_close, uncertainty, confidence)
print_kv("Pred (Target)", extract_target_dict(target))

# 의견(LLM) 생성
opinion = ag.reviewer_draft(ag.stockdata, target)
print_kv("Opinion", extract_opinion_dict(opinion))


In [ ]:

# === DebateAgent 오케스트레이션 테스트 ===
from agents.debate_agent import DebateAgent

debate_agent = DebateAgent(ROUNDS, TICKER)

# (선택) 데이터셋 사전 생성
if hasattr(debate_agent, "run_dataset"):
    try:
        print("[info] run_dataset() 실행")
        debate_agent.run_dataset()
    except Exception as e:
        print(f"[warn] run_dataset() 실패/스킵: {e}")

# Round 0: Opinion
try:
    opinions = debate_agent.get_opinion(0, TICKER, rebuild=REBUILD, force_pretrain=FORCE_PRETRAIN)
    print("=== get_opinion() 결과 ===")
    if opinions:
        for aid, op in opinions.items():
            print_kv(f"Opinion[{aid}]", extract_opinion_dict(op))
    else:
        print("[warn] opinions 비어있음")
except Exception as e:
    print(f"[error] get_opinion 실패: {e}")

# Ensemble (있으면)
if hasattr(debate_agent, "get_ensemble"):
    try:
        ens = debate_agent.get_ensemble()
        if isinstance(ens, dict):
            print_kv("Ensemble", ens)
        else:
            print_kv("Ensemble", {"result": ens})
    except Exception as e:
        print(f"[warn] get_ensemble 실패/스킵: {e}")

# run() (있으면)
if hasattr(debate_agent, "run"):
    try:
        result = debate_agent.run(ticker=TICKER, rounds=ROUNDS, rebuild=REBUILD, force_pretrain=FORCE_PRETRAIN)
        if isinstance(result, dict):
            print_kv("run() result", result)
        else:
            print(f"run() finished: {type(result)}")
    except TypeError:
        # 시그니처가 다르면 기본 호출 재시도
        try:
            result = debate_agent.run()
            if isinstance(result, dict):
                print_kv("run() result", result)
            else:
                print(f"run() finished: {type(result)}")
        except Exception as e:
            print(f"[warn] run() 실패/스킵: {e}")
    except Exception as e:
        print(f"[warn] run() 실패/스킵: {e}")
